In [1]:
import os
import scipy
import numpy as np
import pandas as pd

In [2]:
def get_csv(folder_path):
  csv_files = [f for f in os.listdir(folder_path) if not (f.endswith('5.csv'))]   # NOTE: do not push if all csv files are loaded, load all in local only
  return csv_files

In [3]:
def interpolate_matrix(matrix, scale_factor):
  interpolated_matrix = scipy.ndimage.zoom(matrix, zoom=scale_factor, order=3, mode="grid-wrap")

  return interpolated_matrix

In [ ]:
def shift_energy_matrix(matrix):
    min_energy = np.min(matrix)  # Find the minimum energy value
    shifted_matrix = matrix - min_energy  # Shift all values
    return shifted_matrix

In [ ]:
def scale_energy_matrix(matrix):
    min_energy = np.min(matrix)
    scaled_matrix = 2625.49965 * (matrix - min_energy)
    return scaled_matrix


In [5]:
def scale_energy_matrix(matrix):
    min_energy = np.min(matrix)
    scaled_matrix = 2625.49965 * (matrix - min_energy)
    return scaled_matrix


In [6]:
def normalize_surface(surface):
    """Normalize the energy surface to values between 0 and 1."""
    flat = [e for row in surface for e in row]
    E_min, E_max = min(flat), max(flat)
    if E_max == E_min:
        return [[0.0 for _ in row] for row in surface]  # Avoid division by zero
    return [[(e - E_min) / (E_max - E_min) for e in row] for row in surface]

In [7]:
def csv_to_3d_matrix(folder_path, csv_files):
  
  csv_to_3d_matrix = []

  for csv_file in csv_files:
    energy_levels = pd.read_csv(os.path.join(folder_path, csv_file), header=None, names=['x', 'y', 'Energy Levels'])

    unique_x = energy_levels['x'].unique()
    unique_y = energy_levels['y'].unique()

    x_index = {val: idx for idx, val in enumerate(unique_x)}
    y_index = {val: idx for idx, val in enumerate(unique_y)}

    matrix = np.full((len(unique_y), len(unique_x)), np.nan)

    for _, row in energy_levels.iterrows():
      x_pos = x_index[row['x']]
      y_pos = y_index[row['y']]
      matrix[y_pos, x_pos] = row['Energy Levels']

    interpolated_matrix = interpolate_matrix(matrix=matrix, scale_factor=10)
    scaled_matrix = scale_energy_matrix(interpolated_matrix)

    

    csv_to_3d_matrix.append(scaled_matrix)

  return csv_to_3d_matrix

In [ ]:
def idxtopenalty(idx):
    match idx:
        case 0:
            return "1-1"
        case 1:
            return "1-0.8"
        case 2:
            return "1-0.6"
        case 3:
            return "1-0.4"
        case 4:
            return "1-0.2"
        case 5:
            return "1-0"
        case _:
            print("Default case")